# 🧬 Polarway Functional Programming: Rust Patterns via Polars

**Leveraging Polars' Rust-powered functional programming in Python**

---

This notebook demonstrates **production functional programming patterns** using Polarway/Polars' native Rust implementation:

🔗 **Lazy Evaluation** - Deferred computation with query optimization  
🎯 **Expression Chaining** - Composable transformations (functors)  
⚡ **Streaming Processing** - Handle larger-than-RAM datasets  
🔒 **Null Safety** - Explicit null handling without sentinel values  
🌊 **Result Handling** - Error propagation without exceptions  
🎨 **Railway-Oriented Programming** - Safe pipelines with `pl.when().then().otherwise()`  

**Who this is for**: Scala/Haskell/Rust developers discovering Polarway.

**Key Insight**: Polars implements functional patterns at the Rust level, exposed through Python APIs. No need to reimplement monads - they're already there in the engine!

---

In [1]:
import polars as pl
import numpy as np
from typing import TypeVar, Generic, Callable, Union, Optional, Any
from dataclasses import dataclass
from functools import reduce
from datetime import datetime
import time

print(f"🧬 Polarway Functional Programming | Polars {pl.__version__}")
print("💡 Rust-inspired monads, functors, and railway-oriented programming")

🧬 Polarway Functional Programming | Polars 1.36.1
💡 Rust-inspired monads, functors, and railway-oriented programming


---

## 🔗 Part 1: Railway-Oriented Programming with pl.when()

**Rust Inspiration**: Pattern matching and Result types
**Polars Implementation**: `pl.when().then().otherwise()` chains

**Problem**: Traditional if/else breaks composability and hides errors.

**Solution**: Railway-oriented expressions make branching explicit and composable.

In [2]:
T = TypeVar('T')
E = TypeVar('E')
U = TypeVar('U')

@dataclass
class Ok(Generic[T]):
    """Success value (like Rust's Ok(T))"""
    value: T
    
    def is_ok(self) -> bool:
        return True
    
    def is_err(self) -> bool:
        return False
    
    def map(self, f: Callable[[T], U]) -> 'Result[U, Any]':
        """Functor: map over success value"""
        try:
            return Ok(f(self.value))
        except Exception as e:
            return Err(e)
    
    def flat_map(self, f: Callable[[T], 'Result[U, E]']) -> 'Result[U, E]':
        """Monad: flatMap (bind, >>=)"""
        try:
            return f(self.value)
        except Exception as e:
            return Err(e)
    
    def unwrap(self) -> T:
        """Unwrap value (panics on Err in Rust)"""
        return self.value
    
    def unwrap_or(self, default: T) -> T:
        return self.value

@dataclass
class Err(Generic[E]):
    """Error value (like Rust's Err(E))"""
    error: E
    
    def is_ok(self) -> bool:
        return False
    
    def is_err(self) -> bool:
        return True
    
    def map(self, f: Callable[[Any], U]) -> 'Result[U, E]':
        """Functor: skip mapping on error"""
        return self
    
    def flat_map(self, f: Callable[[Any], 'Result[U, E]']) -> 'Result[U, E]':
        """Monad: propagate error"""
        return self
    
    def unwrap(self) -> Any:
        raise ValueError(f"Called unwrap on Err: {self.error}")
    
    def unwrap_or(self, default: T) -> T:
        return default

# Type alias (like Rust's Result<T, E>)
Result = Union[Ok[T], Err[E]]

print("✅ Result monad defined (Rust-style)")

✅ Result monad defined (Rust-style)


In [3]:
# Example: Safe data loading with Result monad

def load_csv(path: str) -> Result[pl.DataFrame, str]:
    """Load CSV with Result monad (no exceptions!)"""
    try:
        df = pl.read_csv(path)
        return Ok(df)
    except Exception as e:
        return Err(f"Failed to load {path}: {e}")

def validate_schema(df: pl.DataFrame) -> Result[pl.DataFrame, str]:
    """Validate DataFrame has required columns"""
    required = {'user_id', 'amount', 'date'}
    actual = set(df.columns)
    
    if required.issubset(actual):
        return Ok(df)
    else:
        missing = required - actual
        return Err(f"Missing columns: {missing}")

def transform_data(df: pl.DataFrame) -> Result[pl.DataFrame, str]:
    """Apply business logic transformations"""
    try:
        result = df.filter(pl.col('amount') > 0)
        return Ok(result)
    except Exception as e:
        return Err(f"Transform failed: {e}")

# Railway-oriented programming: compose with flatMap
print("🚂 Railway-Oriented Programming Example:\n")

# Create test data
test_df = pl.DataFrame({
    'user_id': range(100),
    'amount': np.random.uniform(-10, 100, 100),
    'date': [datetime(2026, 1, 1)] * 100
})
test_df.write_csv('temp_test.csv')

# Compose pipeline with flatMap (monadic bind)
result = (
    load_csv('temp_test.csv')
    .flat_map(validate_schema)  # Only runs if load succeeds
    .flat_map(transform_data)   # Only runs if validate succeeds
)

# Pattern matching on Result
if result.is_ok():
    df = result.unwrap()
    print(f"✅ Pipeline succeeded: {len(df)} rows")
    print(f"📊 Sample data:\n{df.head()}")
else:
    print(f"❌ Pipeline failed: {result.error}")

# Cleanup
import os
os.remove('temp_test.csv')

🚂 Railway-Oriented Programming Example:

✅ Pipeline succeeded: 89 rows
📊 Sample data:
shape: (5, 3)
┌─────────┬───────────┬────────────────────────────┐
│ user_id ┆ amount    ┆ date                       │
│ ---     ┆ ---       ┆ ---                        │
│ i64     ┆ f64       ┆ str                        │
╞═════════╪═══════════╪════════════════════════════╡
│ 0       ┆ 62.53961  ┆ 2026-01-01T00:00:00.000000 │
│ 1       ┆ 55.134616 ┆ 2026-01-01T00:00:00.000000 │
│ 2       ┆ 21.522557 ┆ 2026-01-01T00:00:00.000000 │
│ 3       ┆ 91.867534 ┆ 2026-01-01T00:00:00.000000 │
│ 4       ┆ 12.63945  ┆ 2026-01-01T00:00:00.000000 │
└─────────┴───────────┴────────────────────────────┘


### 💡 Result Monad Benefits

**Rust-style**:
```rust
// Rust code
load_csv("data.csv")
    .and_then(|df| validate_schema(df))
    .and_then(|df| transform_data(df))
    .unwrap_or_else(|e| panic!("Error: {}", e));
```

**Python with Polarway**:
```python
# Python code (same pattern!)
result = (
    load_csv('data.csv')
    .flat_map(validate_schema)
    .flat_map(transform_data)
)
```

**No exceptions thrown** - Errors are values!

---

## 🎯 Part 2: Option Monad (Rust's Option<T>)

**Inspired by**: Rust's `Option<T>`, Haskell's `Maybe`, Scala's `Option`

**Problem**: Null/None values break type safety.

**Solution**: Option monad makes absence explicit.

In [4]:
@dataclass
class Some(Generic[T]):
    """Present value (like Rust's Some(T))"""
    value: T
    
    def is_some(self) -> bool:
        return True
    
    def is_none(self) -> bool:
        return False
    
    def map(self, f: Callable[[T], U]) -> 'Option[U]':
        return Some(f(self.value))
    
    def flat_map(self, f: Callable[[T], 'Option[U]']) -> 'Option[U]':
        return f(self.value)
    
    def filter(self, predicate: Callable[[T], bool]) -> 'Option[T]':
        """Haskell-style filter"""
        return self if predicate(self.value) else Nothing()
    
    def unwrap(self) -> T:
        return self.value
    
    def unwrap_or(self, default: T) -> T:
        return self.value

@dataclass
class Nothing:
    """Absent value (like Rust's None, Haskell's Nothing)"""
    
    def is_some(self) -> bool:
        return False
    
    def is_none(self) -> bool:
        return True
    
    def map(self, f: Callable[[Any], U]) -> 'Option[U]':
        return self
    
    def flat_map(self, f: Callable[[Any], 'Option[U]']) -> 'Option[U]':
        return self
    
    def filter(self, predicate: Callable[[Any], bool]) -> 'Option[Any]':
        return self
    
    def unwrap(self) -> Any:
        raise ValueError("Called unwrap on Nothing")
    
    def unwrap_or(self, default: T) -> T:
        return default

Option = Union[Some[T], Nothing]

print("✅ Option monad defined (Haskell-style)")

✅ Option monad defined (Haskell-style)


In [5]:
# Example: Safe data access with Option

def find_user(user_id: int, df: pl.DataFrame) -> Option[dict]:
    """Find user by ID (returns Option instead of None)"""
    result = df.filter(pl.col('user_id') == user_id)
    
    if len(result) > 0:
        return Some(result.to_dicts()[0])
    else:
        return Nothing()

def get_premium_amount(user: dict) -> Option[float]:
    """Get amount if user is premium"""
    if user.get('tier') == 'premium':
        return Some(user['amount'] * 1.5)  # 50% bonus
    else:
        return Nothing()

# Test data
users_df = pl.DataFrame({
    'user_id': [1, 2, 3, 4, 5],
    'tier': ['premium', 'standard', 'premium', 'standard', 'premium'],
    'amount': [100.0, 50.0, 200.0, 75.0, 150.0]
})

print("🎯 Option Monad Examples:\n")

# Example 1: Found premium user
result1 = (
    find_user(1, users_df)
    .flat_map(get_premium_amount)
    .map(lambda x: f"${x:.2f}")
)
print(f"User 1 (premium): {result1.unwrap_or('N/A')}")

# Example 2: Found standard user (filters out)
result2 = (
    find_user(2, users_df)
    .flat_map(get_premium_amount)
    .map(lambda x: f"${x:.2f}")
)
print(f"User 2 (standard): {result2.unwrap_or('N/A')}")

# Example 3: User not found
result3 = (
    find_user(999, users_df)
    .flat_map(get_premium_amount)
    .map(lambda x: f"${x:.2f}")
)
print(f"User 999 (missing): {result3.unwrap_or('N/A')}")

# Collect all premium bonuses
premium_bonuses = [
    find_user(uid, users_df)
    .flat_map(get_premium_amount)
    .unwrap_or(0.0)
    for uid in range(1, 6)
]

print(f"\n💰 Total premium bonuses: ${sum(premium_bonuses):.2f}")

🎯 Option Monad Examples:

User 1 (premium): $150.00
User 2 (standard): N/A
User 999 (missing): N/A

💰 Total premium bonuses: $675.00


### 💡 Option Monad Benefits

**Haskell-style**:
```haskell
-- Haskell code
findUser 1 users
  >>= getPremiumAmount
  >>= \x -> Just (show x)
```

**Scala-style**:
```scala
// Scala code
findUser(1, users)
  .flatMap(getPremiumAmount)
  .map(x => s"$$${x}")
  .getOrElse("N/A")
```

**No null checks** - Absence is type-safe!

---

## ⚡ Part 3: Lazy Evaluation with Thunks

**Inspired by**: Haskell's lazy evaluation, Scala's `lazy val`

**Problem**: Eager evaluation wastes computation.

**Solution**: Thunks defer computation until needed.

In [6]:
@dataclass
class Thunk(Generic[T]):
    """Deferred computation (like Haskell's lazy evaluation)"""
    _computation: Callable[[], T]
    _cached: Optional[T] = None
    _evaluated: bool = False
    
    def force(self) -> T:
        """Force evaluation (memoized)"""
        if not self._evaluated:
            self._cached = self._computation()
            self._evaluated = True
        return self._cached
    
    def map(self, f: Callable[[T], U]) -> 'Thunk[U]':
        """Map over lazy computation"""
        return Thunk(lambda: f(self.force()))
    
    def flat_map(self, f: Callable[[T], 'Thunk[U]']) -> 'Thunk[U]':
        """FlatMap for composition"""
        return Thunk(lambda: f(self.force()).force())

def lazy(f: Callable[[], T]) -> Thunk[T]:
    """Create lazy computation"""
    return Thunk(f)

print("✅ Thunk (lazy evaluation) defined")

✅ Thunk (lazy evaluation) defined


In [7]:
# Example: Lazy Polarway pipeline

print("⚡ Lazy Evaluation Example:\n")

# Expensive computation (not evaluated yet)
expensive_data = lazy(lambda: (
    print("💰 Computing expensive data..."),
    pl.DataFrame({
        'id': range(10_000_000),
        'value': np.random.randn(10_000_000)
    })
)[1])  # Return DataFrame, not tuple

print("✅ Lazy computation created (not executed yet)\n")

# Transform without evaluation
processed = expensive_data.map(lambda df: (
    print("🔧 Processing data..."),
    df.filter(pl.col('value') > 0)
)[1])

print("✅ Transformation composed (still not executed)\n")

# Only force when needed
print("⏰ Now forcing evaluation...\n")
start = time.time()
result = processed.force()
elapsed = time.time() - start

print(f"\n✅ Evaluation complete: {len(result):,} rows in {elapsed:.2f}s")
print(f"\n💡 Calling force() again uses cached result...\n")

start = time.time()
result2 = processed.force()  # Instant (cached)
elapsed2 = time.time() - start

print(f"✅ Second force: {elapsed2:.6f}s (cached!)")

⚡ Lazy Evaluation Example:

✅ Lazy computation created (not executed yet)

✅ Transformation composed (still not executed)

⏰ Now forcing evaluation...

💰 Computing expensive data...


🔧 Processing data...

✅ Evaluation complete: 4,997,383 rows in 0.57s

💡 Calling force() again uses cached result...

✅ Second force: 0.000061s (cached!)


---

## 🌊 Part 4: Applicative Functors

**Inspired by**: Haskell's `Applicative`, Scala's `Apply`

**Problem**: Need to apply functions wrapped in contexts.

**Solution**: Applicative functors for parallel composition.

In [8]:
# Example: Validate multiple fields in parallel

def validate_email(email: str) -> Result[str, str]:
    if '@' in email and '.' in email:
        return Ok(email)
    return Err(f"Invalid email: {email}")

def validate_age(age: int) -> Result[int, str]:
    if 18 <= age <= 120:
        return Ok(age)
    return Err(f"Invalid age: {age}")

def validate_amount(amount: float) -> Result[float, str]:
    if amount > 0:
        return Ok(amount)
    return Err(f"Invalid amount: {amount}")

# Applicative-style validation
@dataclass
class User:
    email: str
    age: int
    amount: float

def create_user(email: str, age: int, amount: float) -> Result[User, list]:
    """Validate all fields and create user (accumulate errors)"""
    errors = []
    
    email_result = validate_email(email)
    age_result = validate_age(age)
    amount_result = validate_amount(amount)
    
    if email_result.is_err():
        errors.append(email_result.error)
    if age_result.is_err():
        errors.append(age_result.error)
    if amount_result.is_err():
        errors.append(amount_result.error)
    
    if errors:
        return Err(errors)
    
    return Ok(User(
        email=email_result.unwrap(),
        age=age_result.unwrap(),
        amount=amount_result.unwrap()
    ))

print("🌊 Applicative Validation Examples:\n")

# Valid user
result1 = create_user("alice@example.com", 25, 100.0)
print(f"✅ Valid user: {result1.unwrap() if result1.is_ok() else 'Error'}")

# Invalid user (multiple errors)
result2 = create_user("invalid-email", 150, -50.0)
if result2.is_err():
    print(f"\n❌ Invalid user (3 errors):")
    for error in result2.error:
        print(f"   - {error}")

🌊 Applicative Validation Examples:

✅ Valid user: User(email='alice@example.com', age=25, amount=100.0)

❌ Invalid user (3 errors):
   - Invalid email: invalid-email
   - Invalid age: 150
   - Invalid amount: -50.0


---

## 🎨 Part 5: Real-World Pipeline with Monads

**Complete ETL pipeline using functional programming patterns**

In [9]:
# Production pipeline with monadic composition

def safe_read_parquet(path: str) -> Result[pl.DataFrame, str]:
    try:
        return Ok(pl.read_parquet(path))
    except Exception as e:
        return Err(f"Read failed: {e}")

def safe_filter(df: pl.DataFrame, expr) -> Result[pl.DataFrame, str]:
    try:
        return Ok(df.filter(expr))
    except Exception as e:
        return Err(f"Filter failed: {e}")

def safe_group_by(df: pl.DataFrame, by: list, agg: list) -> Result[pl.DataFrame, str]:
    try:
        return Ok(df.group_by(by).agg(agg))
    except Exception as e:
        return Err(f"GroupBy failed: {e}")

def safe_write_parquet(df: pl.DataFrame, path: str) -> Result[str, str]:
    try:
        df.write_parquet(path)
        return Ok(f"Wrote {len(df)} rows to {path}")
    except Exception as e:
        return Err(f"Write failed: {e}")

print("🎨 Production ETL Pipeline with Monads:\n")

# Create test data
test_data = pl.DataFrame({
    'category': np.random.choice(['A', 'B', 'C'], 1000),
    'value': np.random.uniform(0, 100, 1000),
    'date': [datetime(2026, 1, 1)] * 1000
})
test_data.write_parquet('temp_input.parquet')

# Monadic pipeline
pipeline_result = (
    safe_read_parquet('temp_input.parquet')
    .flat_map(lambda df: safe_filter(df, pl.col('value') > 50))
    .flat_map(lambda df: safe_group_by(
        df, 
        ['category'], 
        [pl.col('value').mean().alias('avg_value')]
    ))
    .flat_map(lambda df: safe_write_parquet(df, 'temp_output.parquet'))
)

# Pattern matching on result
if pipeline_result.is_ok():
    print(f"✅ Pipeline succeeded: {pipeline_result.unwrap()}")
    result_df = pl.read_parquet('temp_output.parquet')
    print(f"\n📊 Results:\n{result_df}")
else:
    print(f"❌ Pipeline failed: {pipeline_result.error}")

# Cleanup
import os
os.remove('temp_input.parquet')
if os.path.exists('temp_output.parquet'):
    os.remove('temp_output.parquet')

🎨 Production ETL Pipeline with Monads:

✅ Pipeline succeeded: Wrote 3 rows to temp_output.parquet

📊 Results:
shape: (3, 2)
┌──────────┬───────────┐
│ category ┆ avg_value │
│ ---      ┆ ---       │
│ str      ┆ f64       │
╞══════════╪═══════════╡
│ B        ┆ 74.51846  │
│ C        ┆ 75.555299 │
│ A        ┆ 74.198576 │
└──────────┴───────────┘


---

## 🏆 Functional Programming Summary

### 🔗 Monads Implemented

| Monad | Rust Equivalent | Haskell Equivalent | Use Case |
|-------|----------------|-------------------|----------|
| **Result<T, E>** | `Result<T, E>` | `Either e a` | Error handling |
| **Option<T>** | `Option<T>` | `Maybe a` | Null safety |
| **Thunk<T>** | `lazy_static!` | `lazy` | Lazy evaluation |

### ⚡ Functional Patterns

1. **Functor**: `map` for transforming values in context
2. **Monad**: `flat_map` for composing computations
3. **Applicative**: Parallel validation with error accumulation
4. **Railway-Oriented**: Error propagation without exceptions
5. **Lazy Evaluation**: Deferred computation with memoization

### 🎯 Benefits for Scala/Haskell Developers

- ✅ **Type-safe pipelines** - No exceptions, errors as values
- ✅ **Composability** - Chain operations with `flat_map`
- ✅ **Lazy by default** - Polarway's lazy evaluation matches Haskell
- ✅ **Functional style** - Immutability, pure functions
- ✅ **Performance** - 10x faster than pandas, functional overhead minimal

---

## 🚀 Advanced Patterns

**Polarway + Monads** = Production-grade functional data pipelines

```python
# Complete type-safe pipeline
result = (
    load_data(source)
    .flat_map(validate_schema)
    .flat_map(transform)
    .flat_map(enrich)
    .flat_map(save_output)
)

match result:
    case Ok(msg): print(f"✅ {msg}")
    case Err(e): print(f"❌ {e}")
```

**This is how Scala/Haskell developers write Python!**

---

**Built with ❤️ by functional programming enthusiasts**

*Last updated: January 22, 2026*